- train : 2011 - 2019
- test : 2020 - 2022

In [170]:
import pandas as pd
import numpy as np

In [171]:
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats

import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib
# matplotlib.rcParams['font.family'] = 'Malgun Gothic'  # windows
matplotlib.rcParams['font.family'] = 'AppleGothic'  # Mac
matplotlib.rcParams['font.size'] = 15  # 글자크기
matplotlib.rcParams['axes.unicode_minus'] = False # 한글 폰트 사용시 마이너스 꺠지지않게 해줌

In [172]:
df = pd.read_csv('./KOSPI+KOSDAQ.csv')
df

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,폐지사유,부실여부
0,(주)DB하이텍,990,2011-12-01,1975-12-12,NaN,1,35.20000,25.50000,5.04000,-42.95000,...,217837.57000,18.17000,29.75000,78.90000,39.45000,62.53000,37.47000,-44.96000,NaN,0
1,(주)DB하이텍,990,2012-12-01,1975-12-12,NaN,1,55.66000,41.39000,7.19000,-21.27000,...,245655.23000,21.22000,37.03000,112.22000,41.58000,55.93000,44.07000,-12.76000,NaN,0
2,(주)DB하이텍,990,2013-12-01,1975-12-12,NaN,1,46.76000,31.58000,7.39000,-23.99000,...,191554.48000,18.10000,31.86000,112.45000,38.80000,67.15000,32.85000,-43.35000,NaN,0
3,(주)DB하이텍,990,2014-12-01,1975-12-12,NaN,1,65.77000,47.39000,10.72000,-13.08000,...,187341.51000,19.02000,33.99000,136.97000,33.00000,68.59000,31.41000,-41.11000,NaN,0
4,(주)DB하이텍,990,2015-12-01,1975-12-12,NaN,1,138.40000,104.20000,44.31000,9.52000,...,386127.30000,39.21000,78.88000,230.24000,57.93000,35.93000,64.07000,32.82000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20382,흥구석유(주),24060,2018-12-01,1994-12-07,NaN,5,329.17000,290.09000,105.67000,11.59000,...,7373.52000,9.80000,19.56000,3064.10000,4.44000,47.47000,52.53000,32.14000,NaN,0
20383,흥구석유(주),24060,2019-12-01,1994-12-07,NaN,5,485.39000,422.76000,49.75000,21.47000,...,15072.74000,18.02000,42.99000,3489.21000,10.00000,22.19000,77.81000,55.02000,NaN,0
20384,흥구석유(주),24060,2020-12-01,1994-12-07,NaN,5,1108.08000,1013.02000,56.54000,30.09000,...,10271.87000,12.27000,34.04000,3280.80000,8.94000,30.41000,69.59000,46.65000,NaN,0
20385,흥구석유(주),24060,2021-12-01,1994-12-07,NaN,5,913.64000,818.58000,31.93000,28.57000,...,5796.91000,7.01000,19.15000,1308.44000,4.38000,53.22000,46.78000,24.13000,NaN,0


In [173]:
# 날짜 데이터 datetime 변경
df["상장일"] = pd.to_datetime(df['상장일'])
df["상장폐지일"] = pd.to_datetime(df['상장폐지일'])
df["회계년도"] = pd.to_datetime(df["회계년도"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20387 entries, 0 to 20386
Data columns (total 58 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   회사명           20387 non-null  object        
 1   거래소코드         20387 non-null  int64         
 2   회계년도          20387 non-null  datetime64[ns]
 3   상장일           20387 non-null  datetime64[ns]
 4   상장폐지일         476 non-null    datetime64[ns]
 5   소속코드          20387 non-null  int64         
 6   유동비율          20387 non-null  float64       
 7   당좌비율          20387 non-null  float64       
 8   현금비율          20387 non-null  float64       
 9   순운전자본비율       20387 non-null  float64       
 10  부채비율          20387 non-null  float64       
 11  자기자본배율        20387 non-null  float64       
 12  이자보상배율(이자비용)  20387 non-null  float64       
 13  비유동비율         20387 non-null  float64       
 14  유동자산구성비율      20387 non-null  float64       
 15  자기자본구성비율      20387 non-null  float6

In [174]:
train = df[~(df['회계년도'].dt.year == 2022) & ~(df['회계년도'].dt.year == 2021) &~(df['회계년도'].dt.year == 2020)]
train

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,폐지사유,부실여부
0,(주)DB하이텍,990,2011-12-01,1975-12-12,NaT,1,35.20000,25.50000,5.04000,-42.95000,...,217837.57000,18.17000,29.75000,78.90000,39.45000,62.53000,37.47000,-44.96000,NaN,0
1,(주)DB하이텍,990,2012-12-01,1975-12-12,NaT,1,55.66000,41.39000,7.19000,-21.27000,...,245655.23000,21.22000,37.03000,112.22000,41.58000,55.93000,44.07000,-12.76000,NaN,0
2,(주)DB하이텍,990,2013-12-01,1975-12-12,NaT,1,46.76000,31.58000,7.39000,-23.99000,...,191554.48000,18.10000,31.86000,112.45000,38.80000,67.15000,32.85000,-43.35000,NaN,0
3,(주)DB하이텍,990,2014-12-01,1975-12-12,NaT,1,65.77000,47.39000,10.72000,-13.08000,...,187341.51000,19.02000,33.99000,136.97000,33.00000,68.59000,31.41000,-41.11000,NaN,0
4,(주)DB하이텍,990,2015-12-01,1975-12-12,NaT,1,138.40000,104.20000,44.31000,9.52000,...,386127.30000,39.21000,78.88000,230.24000,57.93000,35.93000,64.07000,32.82000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20379,흥구석유(주),24060,2015-12-01,1994-12-07,NaT,5,504.11000,443.66000,92.50000,14.15000,...,7835.95000,10.04000,19.44000,3805.60000,5.05000,42.79000,57.21000,37.41000,NaN,0
20380,흥구석유(주),24060,2016-12-01,1994-12-07,NaT,5,411.65000,341.54000,40.08000,14.15000,...,6993.48000,8.84000,18.38000,3684.02000,4.86000,48.08000,51.92000,31.14000,NaN,0
20381,흥구석유(주),24060,2017-12-01,1994-12-07,NaT,5,177.50000,146.69000,17.75000,8.68000,...,6826.79000,8.54000,18.04000,4786.26000,4.39000,47.63000,52.37000,30.24000,NaN,0
20382,흥구석유(주),24060,2018-12-01,1994-12-07,NaT,5,329.17000,290.09000,105.67000,11.59000,...,7373.52000,9.80000,19.56000,3064.10000,4.44000,47.47000,52.53000,32.14000,NaN,0


In [175]:
train['회계년도'].dt.year.value_counts()

회계년도
2019    1884
2018    1788
2017    1707
2016    1618
2015    1538
2014    1480
2013    1442
2012    1417
2011    1361
Name: count, dtype: int64

In [176]:
test = df[(df['회계년도'].dt.year == 2022) | (df['회계년도'].dt.year == 2021) | (df['회계년도'].dt.year == 2020)]
test

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,폐지사유,부실여부
9,(주)DB하이텍,990,2020-12-01,1975-12-12,NaT,1,175.47000,155.66000,18.45000,16.76000,...,483956.34000,41.06000,83.62000,177.32000,51.71000,32.58000,67.42000,34.31000,NaN,0
10,(주)DB하이텍,990,2021-12-01,1975-12-12,NaT,1,243.09000,223.40000,54.58000,30.96000,...,744906.75000,48.06000,134.52000,284.72000,61.33000,24.03000,75.97000,42.54000,NaN,0
11,(주)DB하이텍,990,2022-12-01,1975-12-12,NaT,1,328.36000,310.11000,37.42000,43.50000,...,1068759.34000,50.35000,189.56000,385.52000,63.80000,20.01000,79.99000,52.29000,NaN,0
21,(주)HJ중공업,97230,2020-12-01,2007-08-31,NaT,1,103.19000,87.03000,20.67000,1.50000,...,292475.00000,13.33000,49.32000,1254.23000,17.25000,55.79000,44.21000,26.79000,NaN,0
22,(주)HJ중공업,97230,2021-12-01,2007-08-31,NaT,1,99.90000,84.41000,12.45000,-0.04000,...,109086.00000,4.57000,11.81000,486.17000,6.39000,148.44000,-48.44000,-128.20000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20373,휴젤(주),145020,2021-12-01,2015-12-24,NaT,5,1139.89000,1089.27000,544.21000,55.66000,...,147369.89000,14.59000,200.34000,610.54000,60.11000,30.66000,69.34000,40.98000,NaN,0
20374,휴젤(주),145020,2022-12-01,2015-12-24,NaT,5,348.46000,333.28000,74.32000,41.97000,...,132036.05000,12.58000,186.53000,576.99000,46.88000,40.43000,59.57000,45.96000,NaN,0
20384,흥구석유(주),24060,2020-12-01,1994-12-07,NaT,5,1108.08000,1013.02000,56.54000,30.09000,...,10271.87000,12.27000,34.04000,3280.80000,8.94000,30.41000,69.59000,46.65000,NaN,0
20385,흥구석유(주),24060,2021-12-01,1994-12-07,NaT,5,913.64000,818.58000,31.93000,28.57000,...,5796.91000,7.01000,19.15000,1308.44000,4.38000,53.22000,46.78000,24.13000,NaN,0


In [177]:
test['회계년도'].dt.year.value_counts()

회계년도
2022    2131
2021    2046
2020    1975
Name: count, dtype: int64

- 대략 8:2

In [178]:
# 지수 형태가 아닌 숫자 형태로 표현
pd.options.display.float_format = '{:.5f}'.format

In [179]:
train['이자보상배율(이자비용)'].describe()

count       14235.00000
mean     52898832.96002
std     223837060.19558
min      -4493653.05000
25%             0.39000
50%             4.29000
75%            24.72500
max     999999999.00000
Name: 이자보상배율(이자비용), dtype: float64

In [180]:
# 9999999값을 각 기업 평균 값으로 대체
회사명999 = train[(train['이자보상배율(이자비용)'] == 999999999)]['회사명'].unique()

for i in 회사명999:
    mean = train[(train['회사명'] == i) & (train['이자보상배율(이자비용)'] != 999999999)]['이자보상배율(이자비용)'].mean()

    # 해당 회사명의 '이자보상배율(이자비용)'이 999999999인 값들을 평균값으로 대체
    train.loc[(train['회사명'] == i) & (train['이자보상배율(이자비용)'] == 999999999), '이자보상배율(이자비용)'] = mean

In [181]:
train['이자보상배율(이자비용)'].describe()

count      14188.00000
mean        1512.05762
std       104119.27410
min     -4493653.05000
25%            0.24000
50%            4.00000
75%           20.97000
max      8432098.32000
Name: 이자보상배율(이자비용), dtype: float64

In [182]:
test['이자보상배율(이자비용)'].describe()

count        6152.00000
mean      6339670.17380
std      79374016.97341
min       -355801.07000
25%            -0.79250
50%             4.22000
75%            20.64250
max     999999999.00000
Name: 이자보상배율(이자비용), dtype: float64

In [183]:
# 9999999값을 각 기업 평균 값으로 대체
회사명999 = test[(test['이자보상배율(이자비용)'] == 999999999)]['회사명'].unique()

for i in 회사명999:
    mean = test[(test['회사명'] == i) & (test['이자보상배율(이자비용)'] != 999999999)]['이자보상배율(이자비용)'].mean()

    # 해당 회사명의 '이자보상배율(이자비용)'이 999999999인 값들을 평균값으로 대체
    test.loc[(test['회사명'] == i) & (test['이자보상배율(이자비용)'] == 999999999), '이자보상배율(이자비용)'] = mean

In [184]:
test['이자보상배율(이자비용)'].describe()

count      6136.00000
mean        294.34456
std        9278.42576
min     -355801.07000
25%          -0.82250
50%           4.13500
75%          20.08250
max      522953.43000
Name: 이자보상배율(이자비용), dtype: float64

In [185]:
# 각 기업 평균 값이 없는 경우는 전체 평균 값으로 대체
train['이자보상배율(이자비용)'].fillna(train['이자보상배율(이자비용)'].mean(), inplace = True)

In [186]:
# 각 기업 평균 값이 없는 경우는 전체 평균 값으로 대체
test['이자보상배율(이자비용)'].fillna(test['이자보상배율(이자비용)'].mean(), inplace = True)

### train 이상치 절삭

In [187]:
# 상단 백분위수와 하단 백분위수를 설정
upper_limit = 0.995  # 상단 0.5%를 절삭
lower_limit = 0.005  # 하단 0.5%를 절삭

# 각 변수에 대해 절삭을 적용
for col in train.select_dtypes([np.number]).drop(columns=['부실여부', '거래소코드', '소속코드']).columns:
    # 상단 백분위수를 계산하여 적용
    q_upper = train[col].quantile(upper_limit)
    train[col] = train[col].clip(upper=q_upper)
    
    # 하단 백분위수를 계산하여 적용
    q_lower = train[col].quantile(lower_limit)
    train[col] = train[col].clip(lower=q_lower)

train

,회사명,거래소코드,회계년도,상장일,상장폐지일,소속코드,유동비율,당좌비율,현금비율,순운전자본비율,...,부가가치,총자본투자효율,설비투자효율,기계투자효율,부가가치율,노동소득분배율,자본분배율,이윤분배율,폐지사유,부실여부
0,(주)DB하이텍,990,2011-12-01,1975-12-12,NaT,1,35.20000,25.50000,5.04000,-42.95000,...,217837.57000,18.17000,29.75000,78.90000,39.45000,62.53000,37.47000,-44.96000,NaN,0
1,(주)DB하이텍,990,2012-12-01,1975-12-12,NaT,1,55.66000,41.39000,7.19000,-21.27000,...,245655.23000,21.22000,37.03000,112.22000,41.58000,55.93000,44.07000,-12.76000,NaN,0
2,(주)DB하이텍,990,2013-12-01,1975-12-12,NaT,1,46.76000,31.58000,7.39000,-23.99000,...,191554.48000,18.10000,31.86000,112.45000,38.80000,67.15000,32.85000,-43.35000,NaN,0
3,(주)DB하이텍,990,2014-12-01,1975-12-12,NaT,1,65.77000,47.39000,10.72000,-13.08000,...,187341.51000,19.02000,33.99000,136.97000,33.00000,68.59000,31.41000,-41.11000,NaN,0
4,(주)DB하이텍,990,2015-12-01,1975-12-12,NaT,1,138.40000,104.20000,44.31000,9.52000,...,386127.30000,39.21000,78.88000,230.24000,57.93000,35.93000,64.07000,32.82000,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20379,흥구석유(주),24060,2015-12-01,1994-12-07,NaT,5,504.11000,443.66000,92.50000,14.15000,...,7835.95000,10.04000,19.44000,3805.60000,5.05000,42.79000,57.21000,37.41000,NaN,0
20380,흥구석유(주),24060,2016-12-01,1994-12-07,NaT,5,411.65000,341.54000,40.08000,14.15000,...,6993.48000,8.84000,18.38000,3684.02000,4.86000,48.08000,51.92000,31.14000,NaN,0
20381,흥구석유(주),24060,2017-12-01,1994-12-07,NaT,5,177.50000,146.69000,17.75000,8.68000,...,6826.79000,8.54000,18.04000,4786.26000,4.39000,47.63000,52.37000,30.24000,NaN,0
20382,흥구석유(주),24060,2018-12-01,1994-12-07,NaT,5,329.17000,290.09000,105.67000,11.59000,...,7373.52000,9.80000,19.56000,3064.10000,4.44000,47.47000,52.53000,32.14000,NaN,0


## 최종 피쳐
['매출액순이익률', '자기자본순이익률', '금융비용 대 총비용비율', '매입채무회전기간', '부채비율', '순운전자본비율',
       '재고자산증가율', '총자본사업이익률', '총자본증가율', '총자본투자효율']

In [188]:
# 필요한 열들을 리스트로 저장
selected_columns = ['매출액순이익률', '자기자본순이익률', '금융비용 대 총비용비율', '매입채무회전기간', '부채비율', '순운전자본비율',
       '재고자산증가율', '총자본사업이익률', '총자본증가율', '총자본투자효율', '부실여부']

# 최종피쳐 데이터프레임
train = train[selected_columns]
test = test[selected_columns]

In [189]:
train.describe()

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
count,14235.00000,14235.00000,14235.00000,14235.00000,14235.00000,14235.00000,14235.00000,14235.00000,14235.00000,14235.00000,14235.00000
mean,-3.96163,-1.85694,0.77847,33.25939,116.84503,17.44278,17.07946,3.41592,7.01578,17.15314,0.01138
std,34.73978,28.53021,2.23780,26.26848,152.26214,24.70276,93.52629,8.42115,24.01791,15.66403,0.10607
min,-294.87950,-209.84750,-6.56810,0.00000,1.80680,-51.41220,-89.87640,-36.14290,-54.18560,-55.81200,0.00000
25%,-1.98000,-3.17500,-0.23000,16.24000,35.13500,0.32000,-9.37000,0.63000,-2.27500,10.13000,0.00000
50%,2.48000,4.32000,0.48000,27.79000,76.44000,16.61000,0.25000,3.80000,3.19000,16.89000,0.00000
75%,6.79000,9.98000,1.52000,43.94000,142.33500,33.90000,19.53500,7.50500,11.97000,24.57500,0.00000
max,60.16490,53.94750,12.60810,174.28320,1323.94220,80.02000,951.05200,27.63660,173.66430,83.31470,1.00000


In [190]:
test['부실여부'].value_counts()

부실여부
0    6116
1      36
Name: count, dtype: int64

### feature, target split

In [191]:
# feature & target split

X = train.drop(['부실여부'], axis=1)
y = train['부실여부']

X_test = test.drop(['부실여부'], axis=1)
y_test = test['부실여부']

### Scaling

In [192]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_scaled = scaler.fit_transform(train.drop('부실여부', axis = 1))
test_scaled = scaler.transform(test.drop('부실여부', axis = 1))
X_scaled = pd.DataFrame(data = train_scaled, columns = train.drop('부실여부', axis = 1).columns)
X_test_scaled = pd.DataFrame(data = test_scaled, columns = test.drop('부실여부', axis = 1).columns)
X_scaled

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율
0,-0.39663,-0.90725,3.12440,-0.58892,1.13213,-2.44486,-0.21769,-0.75122,-0.84131,0.06492
1,-0.03881,-0.30190,2.73561,-0.78459,1.18980,-1.56720,-0.15279,-0.52203,-0.43535,0.25964
2,-0.37015,-1.10950,4.11201,-0.59843,1.97072,-1.67731,-0.27554,-0.47690,-0.64936,0.06045
3,-0.27630,-1.59356,4.73765,-0.45910,3.93483,-1.23565,-0.22988,0.14655,-0.58149,0.11919
4,0.66127,1.95605,3.16015,-0.42712,1.12130,-0.32074,0.03807,1.11795,-0.29295,1.40817
...,...,...,...,...,...,...,...,...,...,...
14230,0.16845,0.21791,-0.36576,-1.26618,-0.67284,-0.13330,-0.43304,-0.31183,0.24459,-0.45412
14231,0.15751,0.17620,-0.33895,-1.26618,-0.67284,-0.13330,0.37831,-0.24533,-0.23424,-0.53073
14232,0.15233,0.16919,-0.31660,-1.26351,-0.67179,-0.35474,-0.08201,-0.29046,-0.25131,-0.54989
14233,0.15521,0.18356,-0.35235,-1.26351,-0.72302,-0.23694,-0.67544,-0.22277,-0.53736,-0.46945


In [193]:
X_test_scaled

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율
0,0.62471,0.87373,-0.07082,-0.76556,-0.45000,-0.02764,-0.18241,2.12260,0.05097,1.52628
1,0.86508,1.23407,-0.30766,-0.80249,-0.50385,0.54721,0.10960,3.09043,1.01987,1.97318
2,1.07436,1.49275,-0.88861,-0.88129,-0.57321,1.05487,-0.05709,4.66034,1.24637,2.11938
3,0.24703,1.03637,0.94808,0.26347,3.07852,-0.64541,-0.76056,-0.10402,-0.75804,-0.24408
4,-0.12172,-1.23814,0.60398,0.48732,2.20210,-0.70775,-0.31083,-0.93529,0.07096,-0.80334
...,...,...,...,...,...,...,...,...,...,...
6147,0.82305,0.33078,-0.73220,-0.93611,-0.62437,1.54714,0.01027,0.83057,-0.05978,-0.16364
6148,0.73410,0.32132,-1.95666,-0.97837,-0.61255,0.99293,-0.20037,0.89588,-0.12682,-0.29196
6149,0.23408,0.27961,-0.39704,-1.26465,-0.73340,0.51199,-0.38161,-0.45790,-0.28920,-0.31175
6150,0.14455,0.12713,-0.35235,-1.26465,-0.73622,0.45046,-0.01015,-0.40802,-0.34541,-0.64757


In [194]:
# feature & target split

X = X_scaled
y = train['부실여부']

X_test = X_test_scaled
y_test = test['부실여부']

### ADASYN 2 : 1

In [195]:
from imblearn.over_sampling import ADASYN

# ADASYN 적용
adasyn = ADASYN(sampling_strategy = 0.5, random_state=42)  # 소수 클래스와 다수 클래스의 비율 1:1 만들기
X_resampled, y_resampled = adasyn.fit_resample(X, y)

# 샘플링 후의 데이터셋 확인
resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
resampled_df['부실여부'] = y_resampled
print(resampled_df['부실여부'].value_counts())  # 클래스 간 샘플 개수 확인

부실여부
0    14073
1     7067
Name: count, dtype: int64


In [196]:
resampled_df

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
0,-0.39663,-0.90725,3.12440,-0.58892,1.13213,-2.44486,-0.21769,-0.75122,-0.84131,0.06492,0
1,-0.03881,-0.30190,2.73561,-0.78459,1.18980,-1.56720,-0.15279,-0.52203,-0.43535,0.25964,0
2,-0.37015,-1.10950,4.11201,-0.59843,1.97072,-1.67731,-0.27554,-0.47690,-0.64936,0.06045,0
3,-0.27630,-1.59356,4.73765,-0.45910,3.93483,-1.23565,-0.22988,0.14655,-0.58149,0.11919,0
4,0.66127,1.95605,3.16015,-0.42712,1.12130,-0.32074,0.03807,1.11795,-0.29295,1.40817,0
...,...,...,...,...,...,...,...,...,...,...,...
21135,-0.78699,-2.30595,1.30764,2.95315,1.95688,-1.15826,0.14227,-1.88035,-0.66038,-1.25593,1
21136,-0.63499,-1.30517,0.79063,2.79489,0.97983,-0.77904,-0.20902,-1.84009,-1.59796,-0.81219,1
21137,-0.58116,-1.08617,0.68745,2.69116,0.85198,-0.69646,-0.29335,-1.79858,-1.75418,-0.71172,1
21138,-0.91463,-2.48397,1.26531,3.12296,1.52397,-1.24809,0.18402,-1.91375,-0.81426,-1.49644,1


In [197]:
y_test.values

array([0, 0, 0, ..., 0, 0, 0])

In [198]:
X_test_scaled['부실여부'] = y_test.values

In [199]:
resampled_df

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
0,-0.39663,-0.90725,3.12440,-0.58892,1.13213,-2.44486,-0.21769,-0.75122,-0.84131,0.06492,0
1,-0.03881,-0.30190,2.73561,-0.78459,1.18980,-1.56720,-0.15279,-0.52203,-0.43535,0.25964,0
2,-0.37015,-1.10950,4.11201,-0.59843,1.97072,-1.67731,-0.27554,-0.47690,-0.64936,0.06045,0
3,-0.27630,-1.59356,4.73765,-0.45910,3.93483,-1.23565,-0.22988,0.14655,-0.58149,0.11919,0
4,0.66127,1.95605,3.16015,-0.42712,1.12130,-0.32074,0.03807,1.11795,-0.29295,1.40817,0
...,...,...,...,...,...,...,...,...,...,...,...
21135,-0.78699,-2.30595,1.30764,2.95315,1.95688,-1.15826,0.14227,-1.88035,-0.66038,-1.25593,1
21136,-0.63499,-1.30517,0.79063,2.79489,0.97983,-0.77904,-0.20902,-1.84009,-1.59796,-0.81219,1
21137,-0.58116,-1.08617,0.68745,2.69116,0.85198,-0.69646,-0.29335,-1.79858,-1.75418,-0.71172,1
21138,-0.91463,-2.48397,1.26531,3.12296,1.52397,-1.24809,0.18402,-1.91375,-0.81426,-1.49644,1


In [200]:
X_test_scaled

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
0,0.62471,0.87373,-0.07082,-0.76556,-0.45000,-0.02764,-0.18241,2.12260,0.05097,1.52628,0
1,0.86508,1.23407,-0.30766,-0.80249,-0.50385,0.54721,0.10960,3.09043,1.01987,1.97318,0
2,1.07436,1.49275,-0.88861,-0.88129,-0.57321,1.05487,-0.05709,4.66034,1.24637,2.11938,0
3,0.24703,1.03637,0.94808,0.26347,3.07852,-0.64541,-0.76056,-0.10402,-0.75804,-0.24408,0
4,-0.12172,-1.23814,0.60398,0.48732,2.20210,-0.70775,-0.31083,-0.93529,0.07096,-0.80334,0
...,...,...,...,...,...,...,...,...,...,...,...
6147,0.82305,0.33078,-0.73220,-0.93611,-0.62437,1.54714,0.01027,0.83057,-0.05978,-0.16364,0
6148,0.73410,0.32132,-1.95666,-0.97837,-0.61255,0.99293,-0.20037,0.89588,-0.12682,-0.29196,0
6149,0.23408,0.27961,-0.39704,-1.26465,-0.73340,0.51199,-0.38161,-0.45790,-0.28920,-0.31175,0
6150,0.14455,0.12713,-0.35235,-1.26465,-0.73622,0.45046,-0.01015,-0.40802,-0.34541,-0.64757,0


In [201]:
train = resampled_df
test = X_test_scaled

In [202]:
train.describe()

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
count,21140.00000,21140.00000,21140.00000,21140.00000,21140.00000,21140.00000,21140.00000,21140.00000,21140.00000,21140.00000,21140.00000
mean,-0.54851,-0.62588,0.30459,0.20851,0.32006,-0.26546,-0.10405,-0.45945,-0.23780,-0.44805,0.33430
std,1.81973,1.88948,1.23822,1.23339,1.60299,1.13170,0.88315,1.41859,1.07258,1.47781,0.47175
min,-8.37449,-7.29044,-3.28306,-1.26618,-0.75555,-2.78744,-1.14363,-4.69772,-2.54825,-4.65830,0.00000
25%,-0.52111,-0.76769,-0.35682,-0.58799,-0.50603,-0.96644,-0.38917,-1.05884,-0.64395,-0.89915,0.00000
50%,0.13563,0.11324,0.02092,-0.11076,-0.19096,-0.22558,-0.18262,-0.14439,-0.25672,-0.22237,0.00000
75%,0.27128,0.36340,0.69332,0.63739,0.37266,0.47151,-0.01165,0.34843,0.13175,0.37583,1.00000
max,1.84598,1.95605,5.28647,5.36874,7.92803,2.53330,9.98655,2.87627,6.93876,4.22394,1.00000


In [203]:
test.describe()

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
count,6152.00000,6152.00000,6152.00000,6152.00000,6152.00000,6152.00000,6152.00000,6152.00000,6152.00000,6152.00000,6152.00000
mean,-1.94236,-0.06005,0.03827,-0.01428,-0.00228,0.10887,13.44477,-0.10565,0.13885,-0.09847,0.00585
std,42.28304,1.81287,1.63394,5.18811,2.69980,1.15347,991.39721,1.31409,1.42880,1.40779,0.07628
min,-1872.26575,-76.87606,-73.46693,-1.26618,-0.76742,-27.56966,-1.24846,-24.71435,-4.18143,-40.21414,0.00000
25%,-0.00096,-0.12884,-0.42832,-0.76708,-0.55154,-0.55675,-0.24550,-0.45939,-0.33042,-0.52754,0.00000
50%,0.19522,0.21266,-0.15572,-0.35307,-0.31381,0.08693,-0.13622,-0.01258,-0.10246,-0.05511,0.00000
75%,0.35124,0.44330,0.29563,0.17743,0.07766,0.79669,0.15644,0.44581,0.32630,0.42627,0.00000
max,126.16302,8.03236,13.84066,321.05763,168.22215,3.34214,77734.13382,12.05627,32.33927,12.78815,1.00000


In [204]:
train['부실여부'].value_counts()

부실여부
0    14073
1     7067
Name: count, dtype: int64

In [205]:
test['부실여부'].value_counts()

부실여부
0    6116
1      36
Name: count, dtype: int64

In [206]:
X_test

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
0,0.62471,0.87373,-0.07082,-0.76556,-0.45000,-0.02764,-0.18241,2.12260,0.05097,1.52628,0
1,0.86508,1.23407,-0.30766,-0.80249,-0.50385,0.54721,0.10960,3.09043,1.01987,1.97318,0
2,1.07436,1.49275,-0.88861,-0.88129,-0.57321,1.05487,-0.05709,4.66034,1.24637,2.11938,0
3,0.24703,1.03637,0.94808,0.26347,3.07852,-0.64541,-0.76056,-0.10402,-0.75804,-0.24408,0
4,-0.12172,-1.23814,0.60398,0.48732,2.20210,-0.70775,-0.31083,-0.93529,0.07096,-0.80334,0
...,...,...,...,...,...,...,...,...,...,...,...
6147,0.82305,0.33078,-0.73220,-0.93611,-0.62437,1.54714,0.01027,0.83057,-0.05978,-0.16364,0
6148,0.73410,0.32132,-1.95666,-0.97837,-0.61255,0.99293,-0.20037,0.89588,-0.12682,-0.29196,0
6149,0.23408,0.27961,-0.39704,-1.26465,-0.73340,0.51199,-0.38161,-0.45790,-0.28920,-0.31175,0
6150,0.14455,0.12713,-0.35235,-1.26465,-0.73622,0.45046,-0.01015,-0.40802,-0.34541,-0.64757,0


### UnderSampling 1:3

In [208]:
from imblearn.under_sampling import RandomUnderSampler

# 원본 데이터의 클래스 분포 확인
print("클래스 분포:")
print(X_test['부실여부'].value_counts())

# 0 클래스의 샘플 수와 1 클래스의 샘플 수 계산
num_class_0 = (X_test['부실여부'] == 0).sum()
num_class_1 = (X_test['부실여부'] == 1).sum()

# 0 클래스의 샘플 수를 1 클래스의 3배로 설정
desired_class_0 = num_class_1 * 3

# 언더 샘플링 적용
rus = RandomUnderSampler(sampling_strategy={0: desired_class_0, 1: num_class_1})
X_resampled, y_resampled = rus.fit_resample(X_test.drop(columns=['부실여부']), X_test['부실여부'])

# 샘플링 후의 데이터프레임 생성
resampled_df = pd.DataFrame(X_resampled, columns=X_test.columns.drop('부실여부'))
resampled_df['부실여부'] = y_resampled

# 클래스 간 샘플 개수 확인
print("언더샘플링 후 클래스 분포:")
print(resampled_df['부실여부'].value_counts())

클래스 분포:
부실여부
0    6116
1      36
Name: count, dtype: int64
언더샘플링 후 클래스 분포:
부실여부
0    108
1     36
Name: count, dtype: int64


In [209]:
resampled_df

,매출액순이익률,자기자본순이익률,금융비용 대 총비용비율,매입채무회전기간,부채비율,순운전자본비율,재고자산증가율,총자본사업이익률,총자본증가율,총자본투자효율,부실여부
3637,0.15118,0.12152,-0.25851,-0.32547,-0.49880,1.37832,-0.32013,-0.06839,-0.43576,0.92360,0
5239,-0.09984,-1.24795,0.19284,1.19960,1.87456,-0.74742,-0.36365,-0.19664,-1.02243,0.56098,0
782,-0.11769,-0.20095,0.71123,-0.61709,-0.03130,-0.70775,-0.18262,-1.19298,-0.29212,-0.42284,0
2727,0.05704,-0.10596,0.09006,1.28526,0.46136,-0.96967,-0.09035,-0.31540,-0.39954,0.25453,0
3811,-0.60936,-0.35098,0.22413,0.84441,-0.60723,0.25209,-0.18615,-0.72391,-0.80342,-0.84739,0
...,...,...,...,...,...,...,...,...,...,...,...
3458,0.29626,0.83763,-0.11104,3.04294,1.05496,-0.49967,1.80897,0.25462,1.53783,-0.39539,1
5555,0.47905,0.66307,1.57371,-0.48194,-0.38622,1.30991,0.13954,0.18930,-1.13277,0.53033,1
3720,0.50295,3.31124,-0.18701,-0.60643,-0.17395,-0.36931,-0.70816,-0.06958,-1.56704,2.75845,1
3317,-6.07015,0.06509,1.46199,-1.18775,-0.76742,-3.51361,-0.41337,-1.30936,-2.24323,-6.11634,1


In [210]:
# 데이터 준비
X = train.iloc[:, :-1]
y = train['부실여부']
X_test = resampled_df.iloc[:, :-1]
y_test = resampled_df['부실여부']

## Logistic Regression

### 기본

In [211]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 클래스 가중치 설정 (balanced 옵션 사용)
class_weight = 'balanced'

# 로지스틱 회귀 모델 생성 및 학습
logistic_model = LogisticRegression(class_weight=class_weight)
logistic_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = logistic_model.predict(x_val)

# 성능 평가 - 검증 데이터
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)

# 검증 데이터의 성능 평가 결과 출력
print("Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = logistic_model.predict(X_test)

# 성능 평가 - TEST 데이터
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)

# TEST 데이터의 성능 평가 결과 출력
print("\nTest Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

Validation Set의 성능 평가:
Confusion Matrix:
[[2412  400]
 [ 421  995]]
Accuracy: 0.8058183538315988
Precision: 0.7132616487455197
Recall: 0.702683615819209
F1 Score: 0.7079331198861616
ROC AUC: 0.7802180525753227

Test Set의 성능 평가:
Confusion Matrix:
[[84 24]
 [15 21]]
Accuracy: 0.7291666666666666
Precision: 0.4666666666666667
Recall: 0.5833333333333334
F1 Score: 0.5185185185185186
ROC AUC: 0.6805555555555556


In [212]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 클래스 가중치 설정 (balanced 옵션 사용)
class_weight = 'balanced'

# 로지스틱 회귀 모델 생성 및 학습
logistic_model = LogisticRegression(class_weight=class_weight)
logistic_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = logistic_model.predict(x_val)

# 성능 평가 - 검증 데이터
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)

# 검증 데이터의 성능 평가 결과 출력
print("Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = logistic_model.predict(X_test)

# 성능 평가 - TEST 데이터
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)

# TEST 데이터의 성능 평가 결과 출력
print("\nTest Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

# 임계값 리스트
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 테스트 데이터에 대한 F1 스코어 계산
for threshold in thresholds:
    y_test_pred_threshold = (logistic_model.predict_proba(X_test)[:, 1] > threshold).astype(int)
    f1_test_threshold = f1_score(y_test, y_test_pred_threshold)
    print(f"임계값 {threshold}일 때 Test 데이터의 F1 Score: {f1_test_threshold}")
# 테스트 데이터에 대한 Recall 값 계산
for threshold in thresholds:
    y_test_pred_threshold = (logistic_model.predict_proba(X_test)[:, 1] > threshold).astype(int)
    recall_test_threshold = recall_score(y_test, y_test_pred_threshold)
    print(f"임계값 {threshold}일 때 Test 데이터의 Recall 값: {recall_test_threshold}")

Validation Set의 성능 평가:
Confusion Matrix:
[[2412  400]
 [ 421  995]]
Accuracy: 0.8058183538315988
Precision: 0.7132616487455197
Recall: 0.702683615819209
F1 Score: 0.7079331198861616
ROC AUC: 0.7802180525753227

Test Set의 성능 평가:
Confusion Matrix:
[[84 24]
 [15 21]]
Accuracy: 0.7291666666666666
Precision: 0.4666666666666667
Recall: 0.5833333333333334
F1 Score: 0.5185185185185186
ROC AUC: 0.6805555555555556
임계값 0.3일 때 Test 데이터의 F1 Score: 0.44247787610619466
임계값 0.4일 때 Test 데이터의 F1 Score: 0.5
임계값 0.5일 때 Test 데이터의 F1 Score: 0.5185185185185186
임계값 0.6일 때 Test 데이터의 F1 Score: 0.5
임계값 0.7일 때 Test 데이터의 F1 Score: 0.4999999999999999
임계값 0.3일 때 Test 데이터의 Recall 값: 0.6944444444444444
임계값 0.4일 때 Test 데이터의 Recall 값: 0.6388888888888888
임계값 0.5일 때 Test 데이터의 Recall 값: 0.5833333333333334
임계값 0.6일 때 Test 데이터의 Recall 값: 0.5
임계값 0.7일 때 Test 데이터의 Recall 값: 0.4722222222222222


### Logistic 그리드서치

In [213]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 클래스 가중치 설정 (balanced 옵션 사용)
class_weight = 'balanced'

# 하이퍼파라미터 후보들
param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],  # 규제 강도를 조절하는 파라미터
    'penalty': ['l2', 'l1', 'elasticnet', 'none'],  # 규제 유형
    'max_iter': [1, 5, 10, 100],  # 최대 반복 횟수
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga']  # 옵티마이저 선택
}

# 로지스틱 회귀 모델 생성
logistic_model = LogisticRegression(class_weight=class_weight)

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring='f1')
grid_search.fit(x_train, y_train)

# 최적의 모델과 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 검증 데이터로 예측
y_val_pred = best_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("\nValidation Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = best_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nTest Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

Best Hyperparameters: {'C': 0.01, 'max_iter': 1, 'penalty': 'none', 'solver': 'saga'}

Validation Set의 성능 평가 (GridSearch 이후):
Confusion Matrix:
[[2410  402]
 [ 407 1009]]
Accuracy: 0.8086565752128666
Precision: 0.7150956768249468
Recall: 0.7125706214689266
F1 Score: 0.7138309161655465
ROC AUC: 0.7848059366235102

Test Set의 성능 평가 (GridSearch 이후):
Confusion Matrix:
[[84 24]
 [15 21]]
Accuracy: 0.7291666666666666
Precision: 0.4666666666666667
Recall: 0.5833333333333334
F1 Score: 0.5185185185185186
ROC AUC: 0.6805555555555556


### Logistic - 임계치조절

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 클래스 가중치 설정 (balanced 옵션 사용)
class_weight = 'balanced'

# 로지스틱 회귀 모델 생성 및 학습
logistic_model = LogisticRegression(class_weight=class_weight)

# 하이퍼파라미터 후보들
param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],  # 규제 강도를 조절하는 파라미터
    'penalty': ['l2', 'l1', 'elasticnet', 'none'],  # 규제 유형
    'max_iter': [1, 5, 10, 100],  # 최대 반복 횟수
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga']  # 옵티마이저 선택
}

# 임계값 리스트
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 모델 학습 및 성능 평가
for threshold in thresholds:
    print(f"\n임계값(threshold): {threshold}")
    
    # GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
    grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring='f1')
    grid_search.fit(x_train, y_train)

    # 최적의 모델과 하이퍼파라미터 출력
    print("Best Hyperparameters:", grid_search.best_params_)
    best_model = grid_search.best_estimator_

    # 검증 데이터로 예측
    y_val_pred = best_model.predict_proba(x_val)[:, 1] >= threshold

    # 검증 데이터의 성능 평가 결과 출력
    confusion_mat_val = confusion_matrix(y_val, y_val_pred)
    accuracy_val = accuracy_score(y_val, y_val_pred)
    precision_val = precision_score(y_val, y_val_pred)
    recall_val = recall_score(y_val, y_val_pred)
    f1_val = f1_score(y_val, y_val_pred)
    roc_auc_val = roc_auc_score(y_val, y_val_pred)

    print("Validation Set의 성능 평가 (GridSearch 이후):")
    print("Confusion Matrix:")
    print(confusion_mat_val)
    print("Accuracy:", accuracy_val)
    print("Precision:", precision_val)
    print("Recall:", recall_val)
    print("F1 Score:", f1_val)
    print("ROC AUC:", roc_auc_val)

    # TEST 데이터로 예측
    y_test_pred = best_model.predict_proba(X_test)[:, 1] >= threshold

    # TEST 데이터의 성능 평가 결과 출력
    confusion_mat_test = confusion_matrix(y_test, y_test_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    f1_test = f1_score(y_test, y_test_pred)
    roc_auc_test = roc_auc_score(y_test, y_test_pred)

    print("\nTest Set의 성능 평가 (임계치 조절 이후):")
    print("Confusion Matrix:")
    print(confusion_mat_test)
    print("Accuracy:", accuracy_test)
    print("Precision:", precision_test)
    print("Recall:", recall_test)
    print("F1 Score:", f1_test)
    print("ROC AUC:", roc_auc_test)


임계값(threshold): 0.3
Best Hyperparameters: {'C': 1.0, 'max_iter': 1, 'penalty': 'none', 'solver': 'saga'}
Validation Set의 성능 평가 (GridSearch 이후):
Confusion Matrix:
[[1530 1282]
 [ 225 1191]]
Accuracy: 0.6435666982024598
Precision: 0.48160129397492923
Recall: 0.8411016949152542
F1 Score: 0.6124967858061198
ROC AUC: 0.6925992116112545

Test Set의 성능 평가 (임계치 조절 이후):
Confusion Matrix:
[[263 223]
 [ 22 140]]
Accuracy: 0.6219135802469136
Precision: 0.3856749311294766
Recall: 0.8641975308641975
F1 Score: 0.5333333333333333
ROC AUC: 0.7026748971193415

임계값(threshold): 0.4
Best Hyperparameters: {'C': 10.0, 'max_iter': 1, 'penalty': 'l2', 'solver': 'saga'}
Validation Set의 성능 평가 (GridSearch 이후):
Confusion Matrix:
[[2149  663]
 [ 329 1087]]
Accuracy: 0.7653736991485336
Precision: 0.6211428571428571
Recall: 0.7676553672316384
F1 Score: 0.6866708780795957
ROC AUC: 0.7659400591492472

Test Set의 성능 평가 (임계치 조절 이후):
Confusion Matrix:
[[377 109]
 [ 29 133]]
Accuracy: 0.7870370370370371
Precision: 0.5495867

### Logisitic Stratified K-Fold

In [46]:
# 데이터 준비
X = train.iloc[:, :-1]
y = train['부실여부']
X_test = test.iloc[:, :-1]
y_test = test['부실여부']

In [214]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Stratified K-Fold를 사용하여 데이터를 분할하고 모델 평가를 진행
n_splits = 5
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 모델 성능 지표를 저장할 리스트 초기화
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []

# Stratified K-Fold로 데이터 분할 후 모델 학습과 평가
for train_idx, val_idx in stratified_kf.split(X, y):
    x_train_fold, x_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # 클래스 가중치 설정 (balanced 옵션 사용)
    class_weight = 'balanced'

# 로지스틱 회귀 모델 생성 및 학습
    model = LogisticRegression(class_weight=class_weight)
    model.fit(x_train_fold, y_train_fold)

    # 검증 데이터로 예측
    y_val_pred = model.predict(x_val_fold)

    # 검증 데이터의 성능 평가
    accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
    precision_scores.append(precision_score(y_val_fold, y_val_pred))
    recall_scores.append(recall_score(y_val_fold, y_val_pred))
    f1_scores.append(f1_score(y_val_fold, y_val_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_val_pred))

# Stratified K-Fold 결과 출력
print(f"Stratified K-Fold (n_splits={n_splits}) 결과:")
print(f"평균 Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"평균 Precision: {np.mean(precision_scores):.4f}")
print(f"평균 Recall: {np.mean(recall_scores):.4f}")
print(f"평균 F1 Score: {np.mean(f1_scores):.4f}")
print(f"평균 ROC AUC: {np.mean(roc_auc_scores):.4f}")

# TEST 데이터로 최종 모델 성능 평가
final_model = LogisticRegression()
final_model.fit(X, y)
y_test_pred = final_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\n최종 모델의 TEST Set 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

Stratified K-Fold (n_splits=5) 결과:
평균 Accuracy: 0.8102
평균 Precision: 0.7225
평균 Recall: 0.7020
평균 F1 Score: 0.7120
평균 ROC AUC: 0.7832

최종 모델의 TEST Set 성능 평가:
Confusion Matrix:
[[93 15]
 [18 18]]
Accuracy: 0.7708333333333334
Precision: 0.5454545454545454
Recall: 0.5
F1 Score: 0.5217391304347826
ROC AUC: 0.6805555555555556


### Logistic Stratified K-Fold - GridSearch

In [215]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Stratified K-Fold를 사용하여 데이터를 분할하고 모델 평가를 진행
n_splits = 5
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 하이퍼파라미터 후보들
param_grid = {
    'C': [0.01, 0.1, 1.0, 10.0],  # 규제 강도를 조절하는 파라미터
    'penalty': ['l2', 'l1', 'elasticnet', 'none'],  # 규제 유형
    'max_iter': [1, 5, 10, 100],  # 최대 반복 횟수
    'solver': ['lbfgs', 'liblinear', 'sag', 'saga']  # 옵티마이저 선택
}

# 모델 성능 지표를 저장할 리스트 초기화
f1_scores = []

# Stratified K-Fold로 데이터 분할 후 모델 학습과 평가
for train_idx, val_idx in stratified_kf.split(X, y):
    x_train_fold, x_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # 클래스 가중치 설정 (balanced 옵션 사용)
    class_weight = 'balanced'

    # 로지스틱 회귀 모델 생성
    model = LogisticRegression(class_weight=class_weight)

    # GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1')
    grid_search.fit(x_train_fold, y_train_fold)

    best_model = grid_search.best_estimator_

    # 검증 데이터로 예측
    y_val_pred = best_model.predict(x_val_fold)

    # 검증 데이터의 성능 평가
    f1_scores.append(f1_score(y_val_fold, y_val_pred))

# Stratified K-Fold 결과 출력
print(f"Stratified K-Fold (n_splits={n_splits}) 결과:")
print(f"평균 F1 Score: {np.mean(f1_scores):.4f}")

# TEST 데이터로 최종 모델 성능 평가
final_model = LogisticRegression(**grid_search.best_params_)
final_model.fit(X, y)
y_test_pred = final_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\n최종 모델의 TEST Set 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

Stratified K-Fold (n_splits=5) 결과:
평균 F1 Score: 0.7133

최종 모델의 TEST Set 성능 평가:
Confusion Matrix:
[[91 17]
 [15 21]]
Accuracy: 0.7777777777777778
Precision: 0.5526315789473685
Recall: 0.5833333333333334
F1 Score: 0.5675675675675677
ROC AUC: 0.712962962962963


## LDA

In [216]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LDA 모델 생성
lda_model = LDA()

# 하이퍼파라미터 후보들 (LDA에는 하이퍼파라미터 조정이 없으므로 생략)

# 모델 학습
lda_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = lda_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("LDA Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = lda_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nLDA Test Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

LDA Validation Set의 성능 평가:
Confusion Matrix:
[[2619  193]
 [ 591  825]]
Accuracy: 0.8145695364238411
Precision: 0.8104125736738703
Recall: 0.5826271186440678
F1 Score: 0.6778964667214462
ROC AUC: 0.7569963473732431

LDA Test Set의 성능 평가:
Confusion Matrix:
[[93 15]
 [18 18]]
Accuracy: 0.7708333333333334
Precision: 0.5454545454545454
Recall: 0.5
F1 Score: 0.5217391304347826
ROC AUC: 0.6805555555555556


### LDA StratifiedKFold

In [217]:
from sklearn.model_selection import StratifiedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Stratified K-Fold를 사용하여 데이터를 분할하고 모델 평가를 진행
n_splits = 5
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 모델 성능 지표를 저장할 리스트 초기화
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []

# Stratified K-Fold로 데이터 분할 후 모델 학습과 평가
for train_idx, val_idx in stratified_kf.split(X, y):
    x_train_fold, x_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # 모델 생성 및 학습
    model = LinearDiscriminantAnalysis()
    model.fit(x_train_fold, y_train_fold)

    # 검증 데이터로 예측
    y_val_pred = model.predict(x_val_fold)

    # 검증 데이터의 성능 평가
    accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
    precision_scores.append(precision_score(y_val_fold, y_val_pred))
    recall_scores.append(recall_score(y_val_fold, y_val_pred))
    f1_scores.append(f1_score(y_val_fold, y_val_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_val_pred))

# Stratified K-Fold 결과 출력
print(f"Stratified K-Fold (n_splits={n_splits}) 결과:")
print(f"평균 Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"평균 Precision: {np.mean(precision_scores):.4f}")
print(f"평균 Recall: {np.mean(recall_scores):.4f}")
print(f"평균 F1 Score: {np.mean(f1_scores):.4f}")
print(f"평균 ROC AUC: {np.mean(roc_auc_scores):.4f}")

# TEST 데이터로 최종 모델 성능 평가
final_model = LinearDiscriminantAnalysis()
final_model.fit(X, y)
y_test_pred = final_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\n최종 모델의 TEST Set 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)


Stratified K-Fold (n_splits=5) 결과:
평균 Accuracy: 0.8181
평균 Precision: 0.8244
평균 Recall: 0.5792
평균 F1 Score: 0.6803
평균 ROC AUC: 0.7586

최종 모델의 TEST Set 성능 평가:
Confusion Matrix:
[[93 15]
 [18 18]]
Accuracy: 0.7708333333333334
Precision: 0.5454545454545454
Recall: 0.5
F1 Score: 0.5217391304347826
ROC AUC: 0.6805555555555556


### LDA 임계치 조절

In [218]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LDA 모델 생성 및 학습
lda_model = LDA()
lda_model.fit(x_train, y_train)

# 임계값 리스트
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

for threshold in thresholds:
    print(f"\n임계값(threshold): {threshold}")
    
    # 검증 데이터에 대한 임계값 조정 후의 예측
    y_val_pred_threshold = (lda_model.predict_proba(x_val)[:, 1] > threshold).astype(int)
    
    # 검증 데이터의 성능 평가
    accuracy_val = accuracy_score(y_val, y_val_pred_threshold)
    precision_val = precision_score(y_val, y_val_pred_threshold)
    recall_val = recall_score(y_val, y_val_pred_threshold)
    f1_val = f1_score(y_val, y_val_pred_threshold)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_threshold)
    
    print("Validation Set의 성능 평가 (임계치 조정 이후):")
    print("Accuracy:", accuracy_val)
    print("Precision:", precision_val)
    print("Recall:", recall_val)
    print("F1 Score:", f1_val)
    print("ROC AUC:", roc_auc_val)

    # TEST 데이터에 대한 임계값 조정 후의 예측
    y_test_pred_threshold = (lda_model.predict_proba(X_test)[:, 1] > threshold).astype(int)
    
    # TEST 데이터의 성능 평가
    accuracy_test = accuracy_score(y_test, y_test_pred_threshold)
    precision_test = precision_score(y_test, y_test_pred_threshold)
    recall_test = recall_score(y_test, y_test_pred_threshold)
    f1_test = f1_score(y_test, y_test_pred_threshold)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_threshold)
    
    print("\nTest Set의 성능 평가 (임계치 조정 이후):")
    print("Accuracy:", accuracy_test)
    print("Precision:", precision_test)
    print("Recall:", recall_test)
    print("F1 Score:", f1_test)
    print("ROC AUC:", roc_auc_test)



임계값(threshold): 0.3
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8074739829706717
Precision: 0.7197080291970803
Recall: 0.6963276836158192
F1 Score: 0.7078248384781047
ROC AUC: 0.7798850366869993

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7152777777777778
Precision: 0.4444444444444444
Recall: 0.5555555555555556
F1 Score: 0.49382716049382713
ROC AUC: 0.6620370370370371

임계값(threshold): 0.4
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8122043519394513
Precision: 0.7747349823321554
Recall: 0.6193502824858758
F1 Score: 0.6883830455259027
ROC AUC: 0.7643337472173334

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7569444444444444
Precision: 0.5142857142857142
Recall: 0.5
F1 Score: 0.5070422535211268
ROC AUC: 0.6712962962962963

임계값(threshold): 0.5
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8145695364238411
Precision: 0.8104125736738703
Recall: 0.5826271186440678
F1 Score: 0.6778964667214462
ROC AUC: 0.7569963473732431

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7708333333333334
Precision: 0

### LDA - GridSearch

In [219]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LDA 모델 생성 및 학습
lda_model = LDA(solver='lsqr')
lda_model.fit(x_train, y_train)

# 임계값 리스트
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

for threshold in thresholds:
    print(f"\n임계값(threshold): {threshold}")
    
    # 검증 데이터에 대한 임계값 조정 후의 예측
    y_val_pred_threshold = (lda_model.predict_proba(x_val)[:, 1] > threshold).astype(int)
    
    # 검증 데이터의 성능 평가
    accuracy_val = accuracy_score(y_val, y_val_pred_threshold)
    precision_val = precision_score(y_val, y_val_pred_threshold)
    recall_val = recall_score(y_val, y_val_pred_threshold)
    f1_val = f1_score(y_val, y_val_pred_threshold)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_threshold)
    
    print("Validation Set의 성능 평가 (임계치 조정 이후):")
    print("Accuracy:", accuracy_val)
    print("Precision:", precision_val)
    print("Recall:", recall_val)
    print("F1 Score:", f1_val)
    print("ROC AUC:", roc_auc_val)

    # TEST 데이터에 대한 임계값 조정 후의 예측
    y_test_pred_threshold = (lda_model.predict_proba(X_test)[:, 1] > threshold).astype(int)
    
    # TEST 데이터의 성능 평가
    accuracy_test = accuracy_score(y_test, y_test_pred_threshold)
    precision_test = precision_score(y_test, y_test_pred_threshold)
    recall_test = recall_score(y_test, y_test_pred_threshold)
    f1_test = f1_score(y_test, y_test_pred_threshold)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_threshold)
    
    print("\nTest Set의 성능 평가 (임계치 조정 이후):")
    print("Accuracy:", accuracy_test)
    print("Precision:", precision_test)
    print("Recall:", recall_test)
    print("F1 Score:", f1_test)
    print("ROC AUC:", roc_auc_test)



임계값(threshold): 0.3
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8074739829706717
Precision: 0.7197080291970803
Recall: 0.6963276836158192
F1 Score: 0.7078248384781047
ROC AUC: 0.7798850366869993

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7152777777777778
Precision: 0.4444444444444444
Recall: 0.5555555555555556
F1 Score: 0.49382716049382713
ROC AUC: 0.6620370370370371

임계값(threshold): 0.4
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8122043519394513
Precision: 0.7747349823321554
Recall: 0.6193502824858758
F1 Score: 0.6883830455259027
ROC AUC: 0.7643337472173334

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7569444444444444
Precision: 0.5142857142857142
Recall: 0.5
F1 Score: 0.5070422535211268
ROC AUC: 0.6712962962962963

임계값(threshold): 0.5
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8145695364238411
Precision: 0.8104125736738703
Recall: 0.5826271186440678
F1 Score: 0.6778964667214462
ROC AUC: 0.7569963473732431

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7708333333333334
Precision: 0

## SVM

In [220]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 생성
svm_model = SVC()

# 모델 학습
svm_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = svm_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("SVM Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = svm_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nSVM Test Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

SVM Validation Set의 성능 평가:
Confusion Matrix:
[[2686  126]
 [ 369 1047]]
Accuracy: 0.8829233680227058
Precision: 0.8925831202046036
Recall: 0.739406779661017
F1 Score: 0.8088064889918888
ROC AUC: 0.8472994069002098

SVM Test Set의 성능 평가:
Confusion Matrix:
[[103   5]
 [ 26  10]]
Accuracy: 0.7847222222222222
Precision: 0.6666666666666666
Recall: 0.2777777777777778
F1 Score: 0.39215686274509803
ROC AUC: 0.6157407407407407


### SVM - GridSearchCV

In [221]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 생성 (지정된 하이퍼파라미터 사용)
svm_model = SVC(kernel='rbf', gamma=1, C=0.1, class_weight='balanced')

# 모델 학습
svm_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = svm_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("SVM Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = svm_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nSVM Test Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

SVM Validation Set의 성능 평가:
Confusion Matrix:
[[2374  438]
 [  49 1367]]
Accuracy: 0.8848155156102177
Precision: 0.7573407202216067
Recall: 0.9653954802259888
F1 Score: 0.8488047190313568
ROC AUC: 0.904817228021956

SVM Test Set의 성능 평가:
Confusion Matrix:
[[81 27]
 [ 9 27]]
Accuracy: 0.75
Precision: 0.5
Recall: 0.75
F1 Score: 0.6
ROC AUC: 0.75


### SVM 하이퍼파라미터 조정

In [222]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 생성 (지정된 하이퍼파라미터 사용)
svm_model = SVC(kernel='rbf', gamma=0.1, C=0.1, class_weight='balanced')

# 모델 학습
svm_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = svm_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("SVM Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = svm_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nSVM Test Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

SVM Validation Set의 성능 평가:
Confusion Matrix:
[[2488  324]
 [ 246 1170]]
Accuracy: 0.8651844843897823
Precision: 0.7831325301204819
Recall: 0.826271186440678
F1 Score: 0.8041237113402062
ROC AUC: 0.8555253513995709

SVM Test Set의 성능 평가:
Confusion Matrix:
[[81 27]
 [ 8 28]]
Accuracy: 0.7569444444444444
Precision: 0.509090909090909
Recall: 0.7777777777777778
F1 Score: 0.6153846153846153
ROC AUC: 0.7638888888888888


### SVC - GridSearchCV - 임계치 조절

In [223]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 생성 (지정된 하이퍼파라미터 사용)
svm_model = SVC(kernel='rbf', gamma=0.1, C=0.1, class_weight='balanced')

# 모델 학습
svm_model.fit(x_train, y_train)

# 임계치 리스트
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 각 임계치 별로 예측 및 평가
for threshold in thresholds:
    print(f"\n임계치(threshold): {threshold}")

    # 검증 데이터에 대한 임계치 조정 후의 예측
    y_val_pred_threshold = (svm_model.decision_function(x_val) > threshold).astype(int)
    
    # 검증 데이터의 성능 평가
    confusion_mat_val = confusion_matrix(y_val, y_val_pred_threshold)
    accuracy_val = accuracy_score(y_val, y_val_pred_threshold)
    precision_val = precision_score(y_val, y_val_pred_threshold)
    recall_val = recall_score(y_val, y_val_pred_threshold)
    f1_val = f1_score(y_val, y_val_pred_threshold)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_threshold)

    print("Validation Set의 성능 평가 (임계치 조정 이후):")
    print("Confusion Matrix:")
    print(confusion_mat_val)
    print("Accuracy:", accuracy_val)
    print("Precision:", precision_val)
    print("Recall:", recall_val)
    print("F1 Score:", f1_val)
    print("ROC AUC:", roc_auc_val)

    # TEST 데이터에 대한 임계치 조정 후의 예측
    y_test_pred_threshold = (svm_model.decision_function(X_test) > threshold).astype(int)

    # TEST 데이터의 성능 평가
    confusion_mat_test = confusion_matrix(y_test, y_test_pred_threshold)
    accuracy_test = accuracy_score(y_test, y_test_pred_threshold)
    precision_test = precision_score(y_test, y_test_pred_threshold)
    recall_test = recall_score(y_test, y_test_pred_threshold)
    f1_test = f1_score(y_test, y_test_pred_threshold)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_threshold)

    print("\nTest Set의 성능 평가 (임계치 조정 이후):")
    print("Confusion Matrix:")
    print(confusion_mat_test)
    print("Accuracy:", accuracy_test)
    print("Precision:", precision_test)
    print("Recall:", recall_test)
    print("F1 Score:", f1_test)
    print("ROC AUC:", roc_auc_test)


임계치(threshold): 0.3
Validation Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[2607  205]
 [ 306 1110]]
Accuracy: 0.8791390728476821
Precision: 0.844106463878327
Recall: 0.7838983050847458
F1 Score: 0.8128890516294398
ROC AUC: 0.8554982279335536

Test Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[96 12]
 [23 13]]
Accuracy: 0.7569444444444444
Precision: 0.52
Recall: 0.3611111111111111
F1 Score: 0.42622950819672134
ROC AUC: 0.625

임계치(threshold): 0.4
Validation Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[2631  181]
 [ 325 1091]]
Accuracy: 0.880321665089877
Precision: 0.8577044025157232
Recall: 0.7704802259887006
F1 Score: 0.8117559523809524
ROC AUC: 0.8530566137055879

Test Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[99  9]
 [23 13]]
Accuracy: 0.7777777777777778
Precision: 0.5909090909090909
Recall: 0.3611111111111111
F1 Score: 0.4482758620689656
ROC AUC: 0.6388888888888888

임계치(threshold): 0.5
Validation Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[2650  162]
 [ 341 1075]]
Accuracy: 0.8810312

### SVC - 임계치 조절

In [224]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 생성
svm_model = SVC(class_weight='balanced')  # 'class_weight' 파라미터 추가
svm_model.fit(x_train, y_train)

# 임계치 리스트
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 각 임계치값별로 성능 평가
for threshold in thresholds:
    print(f"\n임계치(threshold): {threshold}")
    
    # 검증 데이터 예측 확률 계산
    y_val_pred_prob = svm_model.decision_function(x_val)
    
    # 임계치에 따른 예측 조정
    y_val_pred_threshold = (y_val_pred_prob > threshold).astype(int)
    
    # 검증 데이터의 성능 평가 결과 계산
    accuracy_val = accuracy_score(y_val, y_val_pred_threshold)
    precision_val = precision_score(y_val, y_val_pred_threshold)
    recall_val = recall_score(y_val, y_val_pred_threshold)
    f1_val = f1_score(y_val, y_val_pred_threshold)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_threshold)
    
    print("Validation Set의 성능 평가 (임계치 조정 이후):")
    print("Accuracy:", accuracy_val)
    print("Precision:", precision_val)
    print("Recall:", recall_val)
    print("F1 Score:", f1_val)
    print("ROC AUC:", roc_auc_val)
    
    # TEST 데이터 예측 확률 계산
    y_test_pred_prob = svm_model.decision_function(X_test)
    
    # 임계치에 따른 예측 조정
    y_test_pred_threshold = (y_test_pred_prob > threshold).astype(int)
    
    # TEST 데이터의 성능 평가 결과 계산
    accuracy_test = accuracy_score(y_test, y_test_pred_threshold)
    precision_test = precision_score(y_test, y_test_pred_threshold)
    recall_test = recall_score(y_test, y_test_pred_threshold)
    f1_test = f1_score(y_test, y_test_pred_threshold)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_threshold)
    
    print("\nTest Set의 성능 평가 (임계치 조정 이후):")
    print("Accuracy:", accuracy_test)
    print("Precision:", precision_test)
    print("Recall:", recall_test)
    print("F1 Score:", f1_test)
    print("ROC AUC:", roc_auc_test)


임계치(threshold): 0.3
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8999526963103122
Precision: 0.8758516275548827
Recall: 0.817090395480226
F1 Score: 0.845451223967848
ROC AUC: 0.8793844580530575

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7569444444444444
Precision: 0.5217391304347826
Recall: 0.3333333333333333
F1 Score: 0.4067796610169491
ROC AUC: 0.6157407407407407

임계치(threshold): 0.4
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.8985335856196783
Precision: 0.8828549262994569
Recall: 0.8036723163841808
F1 Score: 0.8414048059149724
ROC AUC: 0.8749869405533991

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7708333333333334
Precision: 0.5714285714285714
Recall: 0.3333333333333333
F1 Score: 0.4210526315789474
ROC AUC: 0.6249999999999999

임계치(threshold): 0.5
Validation Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.9001892147587511
Precision: 0.8976
Recall: 0.7923728813559322
F1 Score: 0.8417104276069016
ROC AUC: 0.8734268389709959

Test Set의 성능 평가 (임계치 조정 이후):
Accuracy: 0.7708333333333334
Precision: 0

### SVC StratifiedKFold

In [225]:
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Stratified K-Fold를 사용하여 데이터를 분할하고 모델 평가를 진행
n_splits = 5
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# 모델 성능 지표를 저장할 리스트 초기화
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
roc_auc_scores = []

# Stratified K-Fold로 데이터 분할 후 모델 학습과 평가
for train_idx, val_idx in stratified_kf.split(X, y):
    x_train_fold, x_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # 모델 생성 및 학습 (SVM으로 변경)
    model = SVC()
    model.fit(x_train_fold, y_train_fold)

    # 검증 데이터로 예측
    y_val_pred = model.predict(x_val_fold)

    # 검증 데이터의 성능 평가
    accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
    precision_scores.append(precision_score(y_val_fold, y_val_pred))
    recall_scores.append(recall_score(y_val_fold, y_val_pred))
    f1_scores.append(f1_score(y_val_fold, y_val_pred))
    roc_auc_scores.append(roc_auc_score(y_val_fold, y_val_pred))

# Stratified K-Fold 결과 출력
print(f"Stratified K-Fold (n_splits={n_splits}) 결과:")
print(f"평균 Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"평균 Precision: {np.mean(precision_scores):.4f}")
print(f"평균 Recall: {np.mean(recall_scores):.4f}")
print(f"평균 F1 Score: {np.mean(f1_scores):.4f}")
print(f"평균 ROC AUC: {np.mean(roc_auc_scores):.4f}")

# TEST 데이터로 최종 모델 성능 평가
final_model = SVC()
final_model.fit(X, y)
y_test_pred = final_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\n최종 모델의 TEST Set 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

Stratified K-Fold (n_splits=5) 결과:
평균 Accuracy: 0.8801
평균 Precision: 0.8966
평균 Recall: 0.7252
평균 F1 Score: 0.8017
평균 ROC AUC: 0.8416

최종 모델의 TEST Set 성능 평가:
Confusion Matrix:
[[103   5]
 [ 27   9]]
Accuracy: 0.7777777777777778
Precision: 0.6428571428571429
Recall: 0.25
F1 Score: 0.36
ROC AUC: 0.601851851851852


### SVC - 랜덤서치

In [226]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from scipy.stats import reciprocal, uniform

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# SVM 모델 생성
svm_model = SVC(class_weight='balanced')

# 탐색할 하이퍼파라미터 분포 설정
param_dist = {
    'kernel': ['linear', 'rbf'],
    'C': reciprocal(0.1, 10),
    'gamma': uniform(0.01, 1)
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(svm_model, param_distributions=param_dist, n_iter=10, scoring='roc_auc', cv=3, random_state=42, verbose=2, n_jobs=-1)

# 랜덤 서치 수행
random_search.fit(x_train, y_train)

# 최적의 모델로 재학습
best_svm_model = random_search.best_estimator_
best_svm_model.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", random_search.best_params_)

# 임계치 리스트
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 각 임계치 별로 예측 및 평가
for threshold in thresholds:
    print(f"\n임계치(threshold): {threshold}")
    
    # 검증 데이터에 대한 임계치 조정 후의 예측
    y_val_pred_threshold = (best_svm_model.decision_function(x_val) > threshold).astype(int)
    
    # 검증 데이터의 성능 평가
    confusion_mat_val = confusion_matrix(y_val, y_val_pred_threshold)
    accuracy_val = accuracy_score(y_val, y_val_pred_threshold)
    precision_val = precision_score(y_val, y_val_pred_threshold)
    recall_val = recall_score(y_val, y_val_pred_threshold)
    f1_val = f1_score(y_val, y_val_pred_threshold)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_threshold)

    print("Validation Set의 성능 평가 (임계치 조정 이후):")
    print("Confusion Matrix:")
    print(confusion_mat_val)
    print("Accuracy:", accuracy_val)
    print("Precision:", precision_val)
    print("Recall:", recall_val)
    print("F1 Score:", f1_val)
    print("ROC AUC:", roc_auc_val)

    # TEST 데이터에 대한 임계치 조정 후의 예측
    y_test_pred_threshold = (best_svm_model.decision_function(X_test) > threshold).astype(int)

    # TEST 데이터의 성능 평가
    confusion_mat_test = confusion_matrix(y_test, y_test_pred_threshold)
    accuracy_test = accuracy_score(y_test, y_test_pred_threshold)
    precision_test = precision_score(y_test, y_test_pred_threshold)
    recall_test = recall_score(y_test, y_test_pred_threshold)
    f1_test = f1_score(y_test, y_test_pred_threshold)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_threshold)

    print("\nTest Set의 성능 평가 (임계치 조정 이후):")
    print("Confusion Matrix:")
    print(confusion_mat_test)
    print("Accuracy:", accuracy_test)
    print("Precision:", precision_test)
    print("Recall:", recall_test)
    print("F1 Score:", f1_test)
    print("ROC AUC:", roc_auc_test)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
최적의 하이퍼파라미터: {'C': 3.6256176345762263, 'gamma': 0.606850157946487, 'kernel': 'rbf'}

임계치(threshold): 0.3
Validation Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[2790   22]
 [  23 1393]]
Accuracy: 0.989356669820246
Precision: 0.984452296819788
Recall: 0.9837570621468926
F1 Score: 0.9841045566937477
ROC AUC: 0.9879667245300608

Test Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[108   0]
 [ 36   0]]
Accuracy: 0.75
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
ROC AUC: 0.5

임계치(threshold): 0.4
Validation Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[2796   16]
 [  29 1387]]
Accuracy: 0.989356669820246
Precision: 0.9885958660014256
Recall: 0.9795197740112994
F1 Score: 0.9840368925150763
ROC AUC: 0.9869149367922784

Test Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[108   0]
 [ 36   0]]
Accuracy: 0.75
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
ROC AUC: 0.5

임계치(threshold): 0.5
Validation Set의 성능 평가 (임계치 조정 이후):
Confusion Matrix:
[[2800   12]
 [ 

## Random Forest

In [227]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델 생성
rf_model = RandomForestClassifier()

# 모델 학습
rf_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = rf_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("Random Forest Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = rf_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nRandom Forest Test Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

Random Forest Validation Set의 성능 평가:
Confusion Matrix:
[[2779   33]
 [  40 1376]]
Accuracy: 0.9827341532639546
Precision: 0.9765791341376863
Recall: 0.9717514124293786
F1 Score: 0.9741592920353982
ROC AUC: 0.980007996399611

Random Forest Test Set의 성능 평가:
Confusion Matrix:
[[108   0]
 [ 27   9]]
Accuracy: 0.8125
Precision: 1.0
Recall: 0.25
F1 Score: 0.4
ROC AUC: 0.625


### Random Forest 하이퍼파라미터 조정

In [228]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델 생성 (하이퍼파라미터 포함)
rf_model = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=4, min_samples_split=7, min_samples_leaf=5, max_features='sqrt')

# 모델 학습
rf_model.fit(x_train, y_train)

# 각 임계치별 평가를 저장할 딕셔너리 초기화
threshold_evaluations = {}

# 원하는 임계치 범위
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 각 임계치별로 평가 수행
for threshold in thresholds:
    y_val_pred_thresh = (rf_model.predict_proba(x_val)[:, 1] >= threshold).astype(int)
    y_test_pred_thresh = (rf_model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
    
    val_metrics = {
        'Accuracy': accuracy_score(y_val, y_val_pred_thresh),
        'Precision': precision_score(y_val, y_val_pred_thresh),
        'Recall': recall_score(y_val, y_val_pred_thresh),
        'F1 Score': f1_score(y_val, y_val_pred_thresh),
        'ROC AUC': roc_auc_score(y_val, y_val_pred_thresh)
    }
    
    test_metrics = {
        'Accuracy': accuracy_score(y_test, y_test_pred_thresh),
        'Precision': precision_score(y_test, y_test_pred_thresh),
        'Recall': recall_score(y_test, y_test_pred_thresh),
        'F1 Score': f1_score(y_test, y_test_pred_thresh),
        'ROC AUC': roc_auc_score(y_test, y_test_pred_thresh)
    }
    
    threshold_evaluations[threshold] = {
        'Validation Metrics': val_metrics,
        'Test Metrics': test_metrics
    }

# 각 임계치별 평가 결과 출력
for threshold, metrics in threshold_evaluations.items():
    print(f"\n임계치: {threshold}")
    
    print("Validation Metrics:")
    for metric, value in metrics['Validation Metrics'].items():
        print(f"{metric}: {value}")
        
    print("Test Metrics:")
    for metric, value in metrics['Test Metrics'].items():
        print(f"{metric}: {value}")



임계치: 0.3
Validation Metrics:
Accuracy: 0.8195364238410596
Precision: 0.7096981374438022
Recall: 0.780367231638418
F1 Score: 0.743356878573831
ROC AUC: 0.8098137722914709
Test Metrics:
Accuracy: 0.7430555555555556
Precision: 0.49056603773584906
Recall: 0.7222222222222222
F1 Score: 0.5842696629213483
ROC AUC: 0.7361111111111112

임계치: 0.4
Validation Metrics:
Accuracy: 0.8292336802270577
Precision: 0.7518142235123367
Recall: 0.731638418079096
F1 Score: 0.7415891195418755
ROC AUC: 0.8050083982287373
Test Metrics:
Accuracy: 0.7638888888888888
Precision: 0.5217391304347826
Recall: 0.6666666666666666
F1 Score: 0.5853658536585366
ROC AUC: 0.7314814814814814

임계치: 0.5
Validation Metrics:
Accuracy: 0.8398770104068117
Precision: 0.7967871485943775
Recall: 0.7005649717514124
F1 Score: 0.7455843667794062
ROC AUC: 0.8052967106267731
Test Metrics:
Accuracy: 0.8055555555555556
Precision: 0.6176470588235294
Recall: 0.5833333333333334
F1 Score: 0.6
ROC AUC: 0.7314814814814815

임계치: 0.6
Validation Metric

### Random Forest 그리드 서치

In [230]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성
rf_model = RandomForestClassifier()

# 탐색할 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# 그리드 서치 객체 생성
grid_search = GridSearchCV(rf_model, param_grid=param_grid, scoring='roc_auc', cv=3, verbose=2, n_jobs=-1)

# 그리드 서치 수행
grid_search.fit(x_train, y_train)

# 최적의 모델로 재학습
best_rf_model = grid_search.best_estimator_
best_rf_model.fit(x_train, y_train)

# 각 임계치별 평가를 저장할 딕셔너리 초기화
threshold_evaluations = {}

# 원하는 임계치 범위
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 각 임계치별로 평가 수행
for threshold in thresholds:
    y_val_pred_thresh = (best_rf_model.predict_proba(x_val)[:, 1] >= threshold).astype(int)
    y_test_pred_thresh = (best_rf_model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
    
    val_metrics = {
        'Accuracy': accuracy_score(y_val, y_val_pred_thresh),
        'Precision': precision_score(y_val, y_val_pred_thresh),
        'Recall': recall_score(y_val, y_val_pred_thresh),
        'F1 Score': f1_score(y_val, y_val_pred_thresh),
        'ROC AUC': roc_auc_score(y_val, y_val_pred_thresh)
    }
    
    test_metrics = {
        'Accuracy': accuracy_score(y_test, y_test_pred_thresh),
        'Precision': precision_score(y_test, y_test_pred_thresh),
        'Recall': recall_score(y_test, y_test_pred_thresh),
        'F1 Score': f1_score(y_test, y_test_pred_thresh),
        'ROC AUC': roc_auc_score(y_test, y_test_pred_thresh)
    }
    
    threshold_evaluations[threshold] = {
        'Validation Metrics': val_metrics,
        'Test Metrics': test_metrics
    }

# 각 임계치별 평가 결과 출력
for threshold, metrics in threshold_evaluations.items():
    print(f"\n임계치: {threshold}")
    
    print("Validation Metrics:")
    for metric, value in metrics['Validation Metrics'].items():
        print(f"{metric}: {value}")
        
    print("Test Metrics:")
    for metric, value in metrics['Test Metrics'].items():
        print(f"{metric}: {value}")


Fitting 3 folds for each of 243 candidates, totalling 729 fits

임계치: 0.3
Validation Metrics:
Accuracy: 0.9709082308420057
Precision: 0.9239344262295082
Recall: 0.9950564971751412
F1 Score: 0.9581774906494389
ROC AUC: 0.976902359540629
Test Metrics:
Accuracy: 0.8263888888888888
Precision: 0.72
Recall: 0.5
F1 Score: 0.5901639344262295
ROC AUC: 0.7175925925925927

임계치: 0.4
Validation Metrics:
Accuracy: 0.9829706717123936
Precision: 0.9621733149931224
Recall: 0.9879943502824858
F1 Score: 0.9749128919860627
ROC AUC: 0.9842176587827791
Test Metrics:
Accuracy: 0.8402777777777778
Precision: 0.9333333333333333
Recall: 0.3888888888888889
F1 Score: 0.5490196078431373
ROC AUC: 0.6898148148148148

임계치: 0.5
Validation Metrics:
Accuracy: 0.9858088930936613
Precision: 0.9788135593220338
Recall: 0.9788135593220338
F1 Score: 0.9788135593220338
ROC AUC: 0.9840724980109459
Test Metrics:
Accuracy: 0.8263888888888888
Precision: 1.0
Recall: 0.3055555555555556
F1 Score: 0.46808510638297873
ROC AUC: 0.65277777

### Random Forest RandomSearchCV

In [229]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from scipy.stats import randint, uniform

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 생성
rf_model = RandomForestClassifier()

# 탐색할 하이퍼파라미터 분포 설정
param_dist = {
    'n_estimators': randint(10, 200),
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2']
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(rf_model, param_distributions=param_dist, n_iter=10, scoring='roc_auc', cv=3, random_state=42, verbose=2, n_jobs=-1)

# 랜덤 서치 수행
random_search.fit(x_train, y_train)

# 최적의 모델로 재학습
best_rf_model = random_search.best_estimator_
best_rf_model.fit(x_train, y_train)

# 최적의 하이퍼파라미터 출력
print("최적의 하이퍼파라미터:", random_search.best_params_)

# 각 임계치별 평가를 저장할 딕셔너리 초기화
threshold_evaluations = {}

# 원하는 임계치 범위
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 각 임계치별로 평가 수행
for threshold in thresholds:
    y_val_pred_thresh = (best_rf_model.predict_proba(x_val)[:, 1] >= threshold).astype(int)
    y_test_pred_thresh = (best_rf_model.predict_proba(X_test)[:, 1] >= threshold).astype(int)
    
    val_metrics = {
        'Accuracy': accuracy_score(y_val, y_val_pred_thresh),
        'Precision': precision_score(y_val, y_val_pred_thresh),
        'Recall': recall_score(y_val, y_val_pred_thresh),
        'F1 Score': f1_score(y_val, y_val_pred_thresh),
        'ROC AUC': roc_auc_score(y_val, y_val_pred_thresh)
    }
    
    test_metrics = {
        'Accuracy': accuracy_score(y_test, y_test_pred_thresh),
        'Precision': precision_score(y_test, y_test_pred_thresh),
        'Recall': recall_score(y_test, y_test_pred_thresh),
        'F1 Score': f1_score(y_test, y_test_pred_thresh),
        'ROC AUC': roc_auc_score(y_test, y_test_pred_thresh)
    }
    
    threshold_evaluations[threshold] = {
        'Validation Metrics': val_metrics,
        'Test Metrics': test_metrics
    }

# 각 임계치별 평가 결과 출력
for threshold, metrics in threshold_evaluations.items():
    print(f"\n임계치: {threshold}")
    
    print("Validation Metrics:")
    for metric, value in metrics['Validation Metrics'].items():
        print(f"{metric}: {value}")
        
    print("Test Metrics:")
    for metric, value in metrics['Test Metrics'].items():
        print(f"{metric}: {value}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
최적의 하이퍼파라미터: {'max_depth': 14, 'max_features': 'sqrt', 'min_samples_leaf': 9, 'min_samples_split': 3, 'n_estimators': 93}

임계치: 0.3
Validation Metrics:
Accuracy: 0.934484389782403
Precision: 0.8561601000625391
Recall: 0.9668079096045198
F1 Score: 0.9081260364842455
ROC AUC: 0.9425077954850479
Test Metrics:
Accuracy: 0.8194444444444444
Precision: 0.6388888888888888
Recall: 0.6388888888888888
F1 Score: 0.6388888888888888
ROC AUC: 0.7592592592592593

임계치: 0.4
Validation Metrics:
Accuracy: 0.9486754966887417
Precision: 0.9081007488087134
Recall: 0.942090395480226
F1 Score: 0.9247833622183709
ROC AUC: 0.9470409303147929
Test Metrics:
Accuracy: 0.8333333333333334
Precision: 0.75
Recall: 0.5
F1 Score: 0.6
ROC AUC: 0.7222222222222221

임계치: 0.5
Validation Metrics:
Accuracy: 0.9519867549668874
Precision: 0.949592290585619
Recall: 0.9046610169491526
F1 Score: 0.9265822784810127
ROC AUC: 0.9402394700677483
Test Metrics:
Accuracy: 0.84027

### Random Forest StratifiedKFold

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest 모델 생성
rf_model = RandomForestClassifier()

# 하이퍼파라미터 후보들
param_grid = {
    'n_estimators': randint(10, 200),
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['auto', 'sqrt', 'log2']
}

# Stratified K-Fold를 사용하여 데이터를 분할
n_splits = 5
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# GridSearchCV를 사용하여 교차 검증과 하이퍼파라미터 튜닝 진행
grid_search = GridSearchCV(rf_model, param_grid, cv=stratified_kf, scoring='f1')
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print("Best Hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 교차 검증 결과 출력
print("교차 검증 결과:")
print("평균 F1 Score:", grid_search.best_score_)

# 검증 데이터로 예측
y_val_pred = best_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("\nValidation Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = best_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nTest Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

## XGBoost

In [232]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 생성
xgb_model = xgb.XGBClassifier()

# 모델 학습
xgb_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = xgb_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("XGBoost Validation Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = xgb_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nXGBoost Test Set의 성능 평가:")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

XGBoost Validation Set의 성능 평가:
Confusion Matrix:
[[2759   53]
 [  30 1386]]
Accuracy: 0.9803689687795648
Precision: 0.9631688672689368
Recall: 0.9788135593220338
F1 Score: 0.9709281961471102
ROC AUC: 0.9799828820792246

XGBoost Test Set의 성능 평가:
Confusion Matrix:
[[107   1]
 [ 30   6]]
Accuracy: 0.7847222222222222
Precision: 0.8571428571428571
Recall: 0.16666666666666666
F1 Score: 0.27906976744186046
ROC AUC: 0.5787037037037036


### XGBoost 하이퍼 파라미터 조정

In [233]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 생성
xgb_model = xgb.XGBClassifier(n_estimators=50, learning_rate=0.05, max_depth=3)

# 모델 학습
xgb_model.fit(x_train, y_train)

# 임계치 후보들
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 임계치별로 성능 평가 결과 저장할 리스트 초기화
results_val = []
results_test = []

# 임계치별로 평가
for threshold in thresholds:
    # 검증 데이터로 예측
    y_val_pred_prob = xgb_model.predict_proba(x_val)[:, 1]
    y_val_pred = (y_val_pred_prob >= threshold).astype(int)

    # 검증 데이터의 성능 평가 결과 계산
    accuracy_val = accuracy_score(y_val, y_val_pred)
    precision_val = precision_score(y_val, y_val_pred)
    recall_val = recall_score(y_val, y_val_pred)
    f1_val = f1_score(y_val, y_val_pred)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_prob)
    
    results_val.append({
        'threshold': threshold,
        'accuracy': accuracy_val,
        'precision': precision_val,
        'recall': recall_val,
        'f1_score': f1_val,
        'roc_auc': roc_auc_val
    })

    # TEST 데이터로 예측
    y_test_pred_prob = xgb_model.predict_proba(X_test)[:, 1]
    y_test_pred = (y_test_pred_prob >= threshold).astype(int)
    
    # TEST 데이터의 성능 평가 결과 계산
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    f1_test = f1_score(y_test, y_test_pred)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_prob)
    
    results_test.append({
        'threshold': threshold,
        'accuracy': accuracy_test,
        'precision': precision_test,
        'recall': recall_test,
        'f1_score': f1_test,
        'roc_auc': roc_auc_test
    })

# 결과 출력
for i in range(len(thresholds)):
    print(f"Threshold: {thresholds[i]}")
    print("Validation Set의 성능 평가:")
    print("Accuracy:", results_val[i]['accuracy'])
    print("Precision:", results_val[i]['precision'])
    print("Recall:", results_val[i]['recall'])
    print("F1 Score:", results_val[i]['f1_score'])
    print("ROC AUC:", results_val[i]['roc_auc'])
    print("Test Set의 성능 평가:")
    print("Accuracy:", results_test[i]['accuracy'])
    print("Precision:", results_test[i]['precision'])
    print("Recall:", results_test[i]['recall'])
    print("F1 Score:", results_test[i]['f1_score'])
    print("ROC AUC:", results_test[i]['roc_auc'])
    print("----------------------------")


Threshold: 0.3
Validation Set의 성능 평가:
Accuracy: 0.8304162724692526
Precision: 0.7221869040050858
Recall: 0.8022598870056498
F1 Score: 0.7601204416192706
ROC AUC: 0.8944020682145125
Test Set의 성능 평가:
Accuracy: 0.7777777777777778
Precision: 0.5384615384615384
Recall: 0.7777777777777778
F1 Score: 0.6363636363636364
ROC AUC: 0.8325617283950618
----------------------------
Threshold: 0.4
Validation Set의 성능 평가:
Accuracy: 0.8495742667928098
Precision: 0.7785714285714286
Recall: 0.769774011299435
F1 Score: 0.7741477272727273
ROC AUC: 0.8944020682145125
Test Set의 성능 평가:
Accuracy: 0.8194444444444444
Precision: 0.631578947368421
Recall: 0.6666666666666666
F1 Score: 0.6486486486486486
ROC AUC: 0.8325617283950618
----------------------------
Threshold: 0.5
Validation Set의 성능 평가:
Accuracy: 0.8429517502365185
Precision: 0.8097199341021417
Recall: 0.6942090395480226
F1 Score: 0.7475285171102662
ROC AUC: 0.8944020682145125
Test Set의 성능 평가:
Accuracy: 0.8125
Precision: 0.6451612903225806
Recall: 0.5555555

### XGBoost 랜덤서치

In [235]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score
from scipy.stats import uniform, randint

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 생성
xgb_model = xgb.XGBClassifier()

# 탐색할 하이퍼파라미터 분포 설정
param_dist = {
    'n_estimators': randint(10, 200),
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(1, 10),
    'min_child_weight': randint(1, 10),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4),
    'gamma': [0, 0.5, 1, 1.5, 2, 5]
}

# 랜덤 서치 객체 생성
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=50, scoring='roc_auc', cv=3, verbose=2, n_jobs=-1, random_state=42)

# 랜덤 서치 수행
random_search.fit(x_train, y_train)

# 최적의 모델로 재학습
best_xgb_model = random_search.best_estimator_
best_xgb_model.fit(x_train, y_train)

# 임계치 후보들
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

# 임계치별로 성능 평가 결과 저장할 리스트 초기화
results_val = []
results_test = []

# 임계치별로 평가
for threshold in thresholds:
    # 검증 데이터로 예측
    y_val_pred_prob = best_xgb_model.predict_proba(x_val)[:, 1]
    y_val_pred = (y_val_pred_prob >= threshold).astype(int)

    # 검증 데이터의 성능 평가 결과 계산
    accuracy_val = accuracy_score(y_val, y_val_pred)
    precision_val = precision_score(y_val, y_val_pred)
    recall_val = recall_score(y_val, y_val_pred)
    f1_val = f1_score(y_val, y_val_pred)
    roc_auc_val = roc_auc_score(y_val, y_val_pred_prob)
    
    results_val.append({
        'threshold': threshold,
        'accuracy': accuracy_val,
        'precision': precision_val,
        'recall': recall_val,
        'f1_score': f1_val,
        'roc_auc': roc_auc_val
    })

    # TEST 데이터로 예측
    y_test_pred_prob = best_xgb_model.predict_proba(X_test)[:, 1]
    y_test_pred = (y_test_pred_prob >= threshold).astype(int)
    
    # TEST 데이터의 성능 평가 결과 계산
    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    f1_test = f1_score(y_test, y_test_pred)
    roc_auc_test = roc_auc_score(y_test, y_test_pred_prob)
    
    results_test.append({
        'threshold': threshold,
        'accuracy': accuracy_test,
        'precision': precision_test,
        'recall': recall_test,
        'f1_score': f1_test,
        'roc_auc': roc_auc_test
    })

# 결과 출력
for i in range(len(thresholds)):
    print(f"Threshold: {thresholds[i]}")
    print("Validation Set의 성능 평가:")
    print("Accuracy:", results_val[i]['accuracy'])
    print("Precision:", results_val[i]['precision'])
    print("Recall:", results_val[i]['recall'])
    print("F1 Score:", results_val[i]['f1_score'])
    print("ROC AUC:", results_val[i]['roc_auc'])
    print("Test Set의 성능 평가:")
    print("Accuracy:", results_test[i]['accuracy'])
    print("Precision:", results_test[i]['precision'])
    print("Recall:", results_test[i]['recall'])
    print("F1 Score:", results_test[i]['f1_score'])
    print("ROC AUC:", results_test[i]['roc_auc'])
    print("----------------------------")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Threshold: 0.3
Validation Set의 성능 평가:
Accuracy: 0.9798959318826869
Precision: 0.950575490859851
Recall: 0.9915254237288136
F1 Score: 0.9706187348772899
ROC AUC: 0.9982196458278082
Test Set의 성능 평가:
Accuracy: 0.8263888888888888
Precision: 0.9230769230769231
Recall: 0.3333333333333333
F1 Score: 0.489795918367347
ROC AUC: 0.7412551440329218
----------------------------
Threshold: 0.4
Validation Set의 성능 평가:
Accuracy: 0.9808420056764428
Precision: 0.9581331503088538
Recall: 0.9858757062146892
F1 Score: 0.9718064740689175
ROC AUC: 0.9982196458278082
Test Set의 성능 평가:
Accuracy: 0.8125
Precision: 0.9090909090909091
Recall: 0.2777777777777778
F1 Score: 0.4255319148936171
ROC AUC: 0.7412551440329218
----------------------------
Threshold: 0.5
Validation Set의 성능 평가:
Accuracy: 0.9822611163670766
Precision: 0.9665970772442589
Recall: 0.9809322033898306
F1 Score: 0.9737118822292323
ROC AUC: 0.9982196458278082
Test Set의 성능 평가:
Accuracy: 0.80

### XGBoost StratifiedKFold

In [234]:
from sklearn.model_selection import train_test_split, StratifiedKFold
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost 모델 생성
xgb_model = xgb.XGBClassifier()

# 하이퍼파라미터 후보들
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [3, 5, 7],  # 트리 최대 깊이
}

# Stratified K-Fold를 사용하여 데이터를 분할
n_splits = 5
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# GridSearchCV를 사용하여 교차 검증과 하이퍼파라미터 튜닝 진행
grid_search = GridSearchCV(xgb_model, param_grid, cv=stratified_kf, scoring='f1')
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print("Best Hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 교차 검증 결과 출력
print("교차 검증 결과:")
print("평균 F1 Score:", grid_search.best_score_)

# 검증 데이터로 예측
y_val_pred = best_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("\nValidation Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = best_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nTest Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

Best Hyperparameters: {'max_depth': 7, 'n_estimators': 200}
교차 검증 결과:
평균 F1 Score: 0.9737091960727581

Validation Set의 성능 평가 (GridSearch 이후):
Confusion Matrix:
[[2775   37]
 [  15 1401]]
Accuracy: 0.9877010406811731
Precision: 0.9742698191933241
Recall: 0.989406779661017
F1 Score: 0.9817799579537491
ROC AUC: 0.9881244424620873

Test Set의 성능 평가 (GridSearch 이후):
Confusion Matrix:
[[108   0]
 [ 27   9]]
Accuracy: 0.8125
Precision: 1.0
Recall: 0.25
F1 Score: 0.4
ROC AUC: 0.625


## LightGBM
- 출력 오류로 코랩에서 진행

In [138]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 생성 (기본 하이퍼파라미터 값 사용)
lgb_model = lgb.LGBMClassifier()

# 모델 학습
lgb_model.fit(x_train, y_train)

# 검증 데이터로 예측
y_val_pred = lgb_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("LightGBM Validation Set의 성능 평가 (기본 하이퍼파라미터 값):")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = lgb_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nLightGBM Test Set의 성능 평가 (기본 하이퍼파라미터 값):")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4855, number of negative: 9554
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000500 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 14409, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.336942 -> initscore=-0.676951
[LightGBM] [Info] Start training from score -0.676951
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

### LightGBM StratifiedKFold

In [139]:
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 생성
lgb_model = lgb.LGBMClassifier()

# 하이퍼파라미터 후보들
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [3, 5, 7],  # 트리 최대 깊이
}

# Stratified K-Fold를 사용하여 데이터를 분할
n_splits = 5
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# GridSearchCV를 사용하여 교차 검증과 하이퍼파라미터 튜닝 진행
grid_search = GridSearchCV(lgb_model, param_grid, cv=stratified_kf, scoring='f1')
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print("Best Hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 교차 검증 결과 출력
print("교차 검증 결과:")
print("평균 F1 Score:", grid_search.best_score_)

# 검증 데이터로 예측
y_val_pred = best_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, y_val_pred)

print("\nValidation Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = best_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, y_test_pred)

print("\nTest Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 4855, number of negative: 9554
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000416 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2549
[LightGBM] [Info] Number of data points in the train set: 14409, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.336942 -> initscore=-0.676951
[LightGBM] [Info] Start training from score -0.676951
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

### LightGBM 랜덤서치

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 생성
lgb_model = lgb.LGBMClassifier()

# 탐색할 하이퍼파라미터 범위 설정
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [15, 31, 63],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# 랜덤서치를 사용하여 최적의 하이퍼파라미터 찾기
random_search = RandomizedSearchCV(lgb_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='f1', random_state=42)
random_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print("Best Hyperparameters:", random_search.best_params_)
best_model = random_search.best_estimator_

# 검증 데이터로 예측
y_val_pred = best_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, best_model.predict_proba(x_val)[:, 1])

print("LightGBM Validation Set의 성능 평가 (RandomizedSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = best_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

print("\nLightGBM Test Set의 성능 평가 (RandomizedSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

### LightGBM 그리드서치

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 모델 생성
lgb_model = lgb.LGBMClassifier()

# 탐색할 하이퍼파라미터 범위 설정
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'num_leaves': [15, 31, 63],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# 그리드서치를 사용하여 최적의 하이퍼파라미터 찾기
grid_search = GridSearchCV(lgb_model, param_grid=param_grid, cv=5, scoring='f1')
grid_search.fit(x_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print("Best Hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_

# 검증 데이터로 예측
y_val_pred = best_model.predict(x_val)

# 검증 데이터의 성능 평가 결과 출력
confusion_mat_val = confusion_matrix(y_val, y_val_pred)
accuracy_val = accuracy_score(y_val, y_val_pred)
precision_val = precision_score(y_val, y_val_pred)
recall_val = recall_score(y_val, y_val_pred)
f1_val = f1_score(y_val, y_val_pred)
roc_auc_val = roc_auc_score(y_val, best_model.predict_proba(x_val)[:, 1])

print("LightGBM Validation Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_val)
print("Accuracy:", accuracy_val)
print("Precision:", precision_val)
print("Recall:", recall_val)
print("F1 Score:", f1_val)
print("ROC AUC:", roc_auc_val)

# TEST 데이터로 예측
y_test_pred = best_model.predict(X_test)

# TEST 데이터의 성능 평가 결과 출력
confusion_mat_test = confusion_matrix(y_test, y_test_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
roc_auc_test = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])

print("\nLightGBM Test Set의 성능 평가 (GridSearch 이후):")
print("Confusion Matrix:")
print(confusion_mat_test)
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("ROC AUC:", roc_auc_test)

___